In [1]:
!pip uninstall -y pylibcudagraph-cu12 rmm-cu12 && pip install  pandas numpy pillow torch torchvision transformers scikit-learn timeout-decorator peft accelerate datasets bitsandbytes bert-score rouge-score rapidfuzz sentence-transformers evaluate

Found existing installation: rmm-cu12 25.2.0
Uninstalling rmm-cu12-25.2.0:
  Successfully uninstalled rmm-cu12-25.2.0
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━

In [2]:
import os
import gc
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
import torch
from transformers import (
    BlipProcessor,
    BlipForQuestionAnswering,
    AutoProcessor,
    Trainer,
    TrainingArguments
)
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model
from tqdm import tqdm
from timeout_decorator import timeout, TimeoutError
from accelerate import Accelerator
from transformers.data.data_collator import default_data_collator
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    f1_score
)
from bert_score import BERTScorer, score
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import evaluate

2025-05-18 06:37:45.617048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747550265.844198      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747550265.906243      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_df = pd.read_csv('/kaggle/input/datasetnewsplit/train.csv')
val_df = pd.read_csv('/kaggle/input/datasetnewsplit/val.csv')
test_df = pd.read_csv('/kaggle/input/datasetnewsplit/test.csv')
print(f"Loaded training dataset with {len(train_df)} entries")
print(f"Loaded validation dataset with {len(val_df)} entries")
print(f"Loaded testing dataset with {len(test_df)} entries")

Loaded training dataset with 80001 entries
Loaded validation dataset with 20001 entries
Loaded testing dataset with 20001 entries


In [4]:
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))

In [5]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import requests

# Load CLIP model from Hugging Face
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [6]:
@timeout(10)  # optional: set timeout if you have the decorator
def predict_answer(image_path, candidate_answers):
    try:
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            return ""
        
        image = Image.open(image_path).convert("RGB")

        inputs = processor(text=candidate_answers, images=image, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)

        top_idx = probs[0].argmax().item()
        predicted_answer = candidate_answers[top_idx]

        del inputs, outputs, logits_per_image, probs
        gc.collect()
        torch.cuda.empty_cache()

        return predicted_answer
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return ""

In [7]:
import os
os.makedirs("/kaggle/working/resultCSV", exist_ok=True)

In [8]:
unique_answers = test_df["answer"].dropna().str.strip().str.lower().unique().tolist()

In [9]:
# Resume support with handling for empty CSV
start_idx = 0
pred_path = '/kaggle/working/resultCSV/predictions.csv'
if os.path.exists(pred_path):
    try:
        existing = pd.read_csv(pred_path)
        if not existing.empty:
            start_idx = len(existing)
            print(f"Resuming from index {start_idx}")
        else:
            print(f"Prediction file {pred_path} is empty, starting from index 0")
    except pd.errors.EmptyDataError:
        print(f"Prediction file {pred_path} is empty, starting from index 0")
else:
    print(f"No existing prediction file found, starting from index 0")

#Lists to store predictions and ground truth
df = test_df
ground_truths = df["answer"].tolist()
predictions = []
y_true = []
y_pred = []
results = []
skipped_entries = []

# Define candidate answers globally
candidate_answers =unique_answers

# Loop through data
for idx, row in tqdm(df.reset_index(drop=True).iterrows(), total=len(df), desc="Processing"):
    if idx < start_idx:
        continue

    full_image_path = row["full_image_path"]
    question = row["question"]  # optional, not used by CLIP directly
    try:
        predicted = predict_answer(full_image_path, candidate_answers)
    except TimeoutError:
        print(f"Timeout processing row {idx}: {full_image_path}")
        skipped_entries.append({"row": idx, "full_image_path": full_image_path, "question": question})
        predicted = ""

    predictions.append(predicted)
    y_true.append(str(ground_truths[idx]).lower())
    y_pred.append(str(predicted).lower())

    results.append({
        "img_path": full_image_path,
        "question": question,
        "true_answer": str(ground_truths[idx]).lower(),
        "predicted_answer": str(predicted).lower()
    })
    if (idx + 1) % 1000 == 0 or (idx + 1) == len(df):
        chunk_df = pd.DataFrame(results)

        if not os.path.exists(pred_path):
            chunk_df.to_csv(pred_path, mode='w', index=False, header=True)
        else:
            chunk_df.to_csv(pred_path, mode='a', index=False, header=False)

        print(f"Checkpoint saved at index {idx + 1}")
        results = []
        y_true = []
        y_pred = []

No existing prediction file found, starting from index 0



Processing:   5%|▍         | 1000/20001 [12:44<4:05:30,  1.29it/s]

Checkpoint saved at index 1000



Processing:  10%|▉         | 2000/20001 [25:39<3:53:26,  1.29it/s]

Checkpoint saved at index 2000


...

Checkpoint saved at index 19000



Processing: 100%|█████████▉| 20000/20001 [4:14:55<00:00,  1.29it/s]

Checkpoint saved at index 20000



Processing: 100%|██████████| 20001/20001 [4:14:56<00:00,  1.31it/s]

Checkpoint saved at index 20001


In [10]:
# Normalize case
preds = pd.read_csv('/kaggle/working/resultCSV/predictions.csv')
preds = preds.dropna()
# preds = preds.iloc[1:].reset_index(drop=True)
all_preds_CLIP = preds['predicted_answer'].tolist()
all_actuals_CLIP = preds['true_answer'].tolist()

In [11]:

preds_l = [p.lower() for p in all_preds_CLIP if isinstance(p, str)]
refs_l = [r.lower() for r in all_actuals_CLIP if isinstance(r, str)]


# Compute exact-match binary metrics
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(all_actuals_CLIP)

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)

# Print metrics
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}")
print(f"Exact-match Recall:    {rec:.3f}")
print(f"Exact-match F1:        {f1:.3f}\n")

# Save predictions and ground truths to CSV
# pred_ref_df = pd.DataFrame({
#     "Prediction": predictions,
#     "Ground_Truth": refs
# })
# df.to_csv('prediction_output.csv', index=False)

Exact-match Accuracy: 0.026
Exact-match Precision: 1.000
Exact-match Recall:    0.026
Exact-match F1:        0.051



In [3]:
from bert_score import score

preds_l = [p.lower() for p in all_preds_CLIP if isinstance(p, str)]
refs_l = [r.lower() for r in all_actuals_CLIP if isinstance(r, str)]
P, R, F1 = score(preds_l, refs_l, lang="en", verbose=True, rescale_with_baseline = True)
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/26 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/313 [00:00<?, ?it/s]

done in 17.16 seconds, 1165.81 sentences/sec
BERTScore - Precision: 0.1364, Recall: 0.5153, F1: 0.3131


In [ ]:

from rapidfuzz.distance import Levenshtein
lev_dists = [Levenshtein.distance(p, r) for p, r in zip(preds_l, refs_l)]
avg_lev = np.mean(lev_dists)
print(f"Levenshtein Distance:")
print(f"Average Edit Distance: {avg_lev:.2f}")

Levenshtein Distance:
Average Edit Distance: 6.16


In [12]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []

for pred, ref in zip(preds_l, refs_l):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

print(f"ROUGE Scores:")
print(f"ROUGE-1 F1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-L F1: {np.mean(rougeL_scores):.4f}")

ROUGE Scores:
ROUGE-1 F1: 0.0299
ROUGE-L F1: 0.0299


In [13]:
# 5. Sentence Embedding Cosine Similarity (memory-efficient pairwise)
model = SentenceTransformer('all-MiniLM-L6-v2')
pred_embeds = model.encode(preds_l, convert_to_tensor=True, device='cuda')
ref_embeds = model.encode(refs_l, convert_to_tensor=True, device='cuda')

cos_sims = [
    util.cos_sim(pred_embeds[i], ref_embeds[i]).item()
    for i in range(len(preds_l))
]

print(f"Sentence-BERT Cosine Similarity:")
print(f"Average Cosine Similarity: {np.mean(cos_sims):.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/626 [00:00<?, ?it/s]

Batches:   0%|          | 0/626 [00:00<?, ?it/s]

Sentence-BERT Cosine Similarity:
Average Cosine Similarity: 0.2732
